<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

**Создание категориальных признаков для каждого пользователя**

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.feather as feather
import pyarrow.compute  as pc
import polars as pl
import numpy as np

In [2]:
read_arrow = feather.read_table('dataset_full.feather')

In [3]:
%%time
data_date_activity = read_arrow.select(['user_id', 'url_host', 'request_cnt', 'date']).\
    group_by(['user_id', 'date']).aggregate([('request_cnt', "sum")])

CPU times: total: 9.22 s
Wall time: 9.57 s


In [4]:
data_date_activity = pl.from_arrow(data_date_activity).\
pivot(index='user_id', columns='date', values='request_cnt_sum', aggregate_function='mean')

In [5]:
data_date_activity.head()

user_id,2022-06-15 00:00:00.000000000,2022-06-19 00:00:00.000000000,2022-06-12 00:00:00.000000000,2022-05-16 00:00:00.000000000,2022-05-30 00:00:00.000000000,2022-03-29 00:00:00.000000000,2022-03-17 00:00:00.000000000,2022-05-19 00:00:00.000000000,2022-06-16 00:00:00.000000000,2022-03-02 00:00:00.000000000,2022-02-28 00:00:00.000000000,2022-02-26 00:00:00.000000000,2022-03-07 00:00:00.000000000,2022-03-24 00:00:00.000000000,2022-03-28 00:00:00.000000000,2022-04-05 00:00:00.000000000,2022-05-02 00:00:00.000000000,2022-03-25 00:00:00.000000000,2022-04-27 00:00:00.000000000,2022-05-31 00:00:00.000000000,2022-03-06 00:00:00.000000000,2022-04-13 00:00:00.000000000,2022-03-30 00:00:00.000000000,2022-04-14 00:00:00.000000000,2022-04-09 00:00:00.000000000,2022-06-11 00:00:00.000000000,2022-03-16 00:00:00.000000000,2022-05-18 00:00:00.000000000,2022-06-18 00:00:00.000000000,2022-06-09 00:00:00.000000000,2022-04-28 00:00:00.000000000,2022-04-10 00:00:00.000000000,2022-06-07 00:00:00.000000000,2022-03-01 00:00:00.000000000,2022-04-16 00:00:00.000000000,2022-05-22 00:00:00.000000000,...,2021-12-21 00:00:00.000000000,2022-01-20 00:00:00.000000000,2022-01-22 00:00:00.000000000,2021-11-10 00:00:00.000000000,2021-12-31 00:00:00.000000000,2022-01-26 00:00:00.000000000,2021-11-09 00:00:00.000000000,2022-01-21 00:00:00.000000000,2021-12-23 00:00:00.000000000,2021-11-14 00:00:00.000000000,2022-10-15 00:00:00.000000000,2022-10-13 00:00:00.000000000,2022-10-09 00:00:00.000000000,2022-10-06 00:00:00.000000000,2022-10-14 00:00:00.000000000,2022-10-12 00:00:00.000000000,2022-10-28 00:00:00.000000000,2022-10-19 00:00:00.000000000,2022-10-07 00:00:00.000000000,2022-10-20 00:00:00.000000000,2022-10-18 00:00:00.000000000,2022-11-01 00:00:00.000000000,2022-10-27 00:00:00.000000000,2022-10-22 00:00:00.000000000,2022-10-31 00:00:00.000000000,2022-10-29 00:00:00.000000000,2022-10-21 00:00:00.000000000,2022-10-26 00:00:00.000000000,2022-10-11 00:00:00.000000000,2022-10-08 00:00:00.000000000,2022-10-16 00:00:00.000000000,2022-10-25 00:00:00.000000000,2022-10-24 00:00:00.000000000,2022-10-30 00:00:00.000000000,2022-10-17 00:00:00.000000000,2022-10-23 00:00:00.000000000,2022-10-10 00:00:00.000000000
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,...,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
45098,26.0,20.0,24.0,29.0,26.0,61.0,7.0,7.0,42.0,32.0,14.0,32.0,33.0,52.0,22.0,17.0,23.0,25.0,37.0,26.0,20.0,18.0,19.0,12.0,31.0,25.0,6.0,10.0,18.0,60.0,12.0,30.0,35.0,10.0,11.0,20.0,...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
117132,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
79395,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
91294,null,3.0,1.0,20.0,4.0,22.0,9.0,22.0,2.0,2.0,63.0,5.0,11.0,14.0,7.0,13.0,5.0,12.0,13.0,1.0,15.0,62.0,31.0,59.0,3.0,8.0,105.0,11.0,25.0,3.0,null,1.0,8.0,38.0,11.0,4.0,...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
161323,null,nul

In [6]:
data_date_activity = data_date_activity.fill_null(0)

In [7]:
for name in data_date_activity.columns[1:]:
    data_date_activity = data_date_activity.with_columns(pl.col(name).cast(pl.Float32))

In [8]:
data_date_activity.head()

user_id,2022-06-15 00:00:00.000000000,2022-06-19 00:00:00.000000000,2022-06-12 00:00:00.000000000,2022-05-16 00:00:00.000000000,2022-05-30 00:00:00.000000000,2022-03-29 00:00:00.000000000,2022-03-17 00:00:00.000000000,2022-05-19 00:00:00.000000000,2022-06-16 00:00:00.000000000,2022-03-02 00:00:00.000000000,2022-02-28 00:00:00.000000000,2022-02-26 00:00:00.000000000,2022-03-07 00:00:00.000000000,2022-03-24 00:00:00.000000000,2022-03-28 00:00:00.000000000,2022-04-05 00:00:00.000000000,2022-05-02 00:00:00.000000000,2022-03-25 00:00:00.000000000,2022-04-27 00:00:00.000000000,2022-05-31 00:00:00.000000000,2022-03-06 00:00:00.000000000,2022-04-13 00:00:00.000000000,2022-03-30 00:00:00.000000000,2022-04-14 00:00:00.000000000,2022-04-09 00:00:00.000000000,2022-06-11 00:00:00.000000000,2022-03-16 00:00:00.000000000,2022-05-18 00:00:00.000000000,2022-06-18 00:00:00.000000000,2022-06-09 00:00:00.000000000,2022-04-28 00:00:00.000000000,2022-04-10 00:00:00.000000000,2022-06-07 00:00:00.000000000,2022-03-01 00:00:00.000000000,2022-04-16 00:00:00.000000000,2022-05-22 00:00:00.000000000,...,2021-12-21 00:00:00.000000000,2022-01-20 00:00:00.000000000,2022-01-22 00:00:00.000000000,2021-11-10 00:00:00.000000000,2021-12-31 00:00:00.000000000,2022-01-26 00:00:00.000000000,2021-11-09 00:00:00.000000000,2022-01-21 00:00:00.000000000,2021-12-23 00:00:00.000000000,2021-11-14 00:00:00.000000000,2022-10-15 00:00:00.000000000,2022-10-13 00:00:00.000000000,2022-10-09 00:00:00.000000000,2022-10-06 00:00:00.000000000,2022-10-14 00:00:00.000000000,2022-10-12 00:00:00.000000000,2022-10-28 00:00:00.000000000,2022-10-19 00:00:00.000000000,2022-10-07 00:00:00.000000000,2022-10-20 00:00:00.000000000,2022-10-18 00:00:00.000000000,2022-11-01 00:00:00.000000000,2022-10-27 00:00:00.000000000,2022-10-22 00:00:00.000000000,2022-10-31 00:00:00.000000000,2022-10-29 00:00:00.000000000,2022-10-21 00:00:00.000000000,2022-10-26 00:00:00.000000000,2022-10-11 00:00:00.000000000,2022-10-08 00:00:00.000000000,2022-10-16 00:00:00.000000000,2022-10-25 00:00:00.000000000,2022-10-24 00:00:00.000000000,2022-10-30 00:00:00.000000000,2022-10-17 00:00:00.000000000,2022-10-23 00:00:00.000000000,2022-10-10 00:00:00.000000000
i32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,...,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
45098,26.0,20.0,24.0,29.0,26.0,61.0,7.0,7.0,42.0,32.0,14.0,32.0,33.0,52.0,22.0,17.0,23.0,25.0,37.0,26.0,20.0,18.0,19.0,12.0,31.0,25.0,6.0,10.0,18.0,60.0,12.0,30.0,35.0,10.0,11.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
117132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91294,0.0,3.0,1.0,20.0,4.0,22.0,9.0,22.0,2.0,2.0,63.0,5.0,11.0,14.0,7.0,13.0,5.0,12.0,13.0,1.0,15.0,62.0,31.0,59.0,3.0,8.0,105.0,11.0,25.0,3.0,0.0,1.0,8.0,38.0,11.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [9]:
cat_list = ['user_id',
            'cpe_manufacturer_name',
            'cpe_type_cd',
            'cpe_model_os_type',
            'price',
           'request_cnt']

In [10]:
data_cat = pl.from_arrow(read_arrow.select(cat_list))

In [11]:
data_cat

user_id,cpe_manufacturer_name,cpe_type_cd,cpe_model_os_type,price,request_cnt
i32,cat,cat,cat,f32,i8
45098,"""Apple""","""smartphone""","""iOS""",20368.0,1
45098,"""Apple""","""smartphone""","""iOS""",20368.0,1
45098,"""Apple""","""smartphone""","""iOS""",20368.0,1
45098,"""Apple""","""smartphone""","""iOS""",20368.0,1
45098,"""Apple""","""smartphone""","""iOS""",20368.0,1
45098,"""Apple""","""smartphone""","""iOS""",20368.0,2
45098,"""Apple""","""smartphone""","""iOS""",20368.0,1
45098,"""Apple""","""smartphone""","""iOS""",20368.0,1
45098,"""Apple""","""smartphone""","""iOS""",20368.0,1


In [12]:
data_cat_pl = data_cat.groupby("user_id").agg([
                                                pl.last('cpe_manufacturer_name'),
                                                pl.last('cpe_type_cd'),
                                                pl.last('cpe_model_os_type'),
                                                pl.last('price'),
                                                pl.col('request_cnt').sum()
                                            ]
)

In [13]:
data_cat_pl.head()

user_id,cpe_manufacturer_name,cpe_type_cd,cpe_model_os_type,price,request_cnt
i32,cat,cat,cat,f32,i64
254752,"""Huawei""","""smartphone""","""Android""",11860.0,350
151184,"""Xiaomi""","""smartphone""","""Android""",20405.0,700
213616,"""Huawei""","""smartphone""","""Android""",8990.0,65
34352,"""Xiaomi""","""smartphone""","""Android""",10221.0,1694
320304,"""Apple""","""smartphone""","""iOS""",116323.0,3026


In [14]:
day_part_pl = pl.from_arrow(read_arrow.select(['user_id', 'part_of_day', 'request_cnt']).\
                            group_by(['user_id', 'part_of_day']).aggregate([('request_cnt', "sum")]))\
                            .pivot(index='user_id', columns='part_of_day', values='request_cnt_sum')

In [15]:
for name in day_part_pl.columns:   
    day_part_pl = day_part_pl.with_columns(pl.col(name).cast(pl.Int32))

In [16]:
day_part_pl.head()

user_id,morning,day,evening,night
i32,i32,i32,i32,i32
45098,610,770,624,257
117132,238,614,430,4
79395,638,1239,1001,428
91294,511,833,836,257
161323,243,99,91,8


In [17]:
day_region_name_pl = pl.from_arrow(read_arrow.select(['user_id', 'region_name', 'request_cnt']).\
                            group_by(['user_id', 'region_name']).aggregate([('request_cnt', "sum")]))\
                            .pivot(index='user_id', columns='region_name', values='request_cnt_sum')

In [18]:
for name in day_region_name_pl.columns:   
    day_region_name_pl = day_region_name_pl.with_columns(pl.col(name).cast(pl.Int32))

In [19]:
day_region_name_pl = day_region_name_pl.fill_null(0)

In [20]:
day_region_name_pl

user_id,Краснодарский край,Ставропольский край,Республика Адыгея,Санкт-Петербург,Калининградская область,Тюменская область,Свердловская область,Ханты-Мансийский АО — Югра,Пермский край,Челябинская область,Мурманская область,Томская область,Красноярский край,Новосибирская область,Кемеровская область - Кузбасс,Республика Алтай,Республика Татарстан,Самарская область,Республика Коми,Кировская область,Республика Марий Эл,Республика Башкортостан,Алтайский край,Тверская область,Хабаровский край,Приморский край,Республика Саха (Якутия),Московская область,Москва,Ярославская область,Орловская область,Ивановская область,Тамбовская область,Ростовская область,Воронежская область,Владимирская область,...,Пензенская область,Ненецкий АО,Новгородская область,Вологодская область,Псковская область,Республика Мордовия,Ульяновская область,Липецкая область,Архангельская область,Омская область,Республика Хакасия,Нижегородская область,Саратовская область,Оренбургская область,Республика Северная Осетия — Алания,Карачаево-Черкесская Республика,Удмуртская Республика,Чувашская Республика,Амурская область,Камчатский край,Курганская область,Брянская область,Белгородская область,Костромская область,Кабардино-Балкарская Республика,Еврейская АО,Республика Калмыкия,Курская область,Чеченская Республика,Астраханская область,Сахалинская область,Республика Ингушетия,Республика Тыва,Ямало-Ненецкий АО,Магаданская область,Чукотский АО,Иркутская область
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,...,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
45098,2254,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
117132,0,0,0,1286,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
79395,0,0,0,3167,139,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
91294,2437,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
161323,0,0,0,0,0,439,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
304574,0,0,0,0,0,0,0,198,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
204489,0,0,0,0,0,0,1014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
148545,0,0,0,0,0,4906,127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16874,0,0,0,0,0,0,0,0,2849,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
day_city_name_pl = pl.from_arrow(read_arrow.select(['user_id', 'city_name', 'request_cnt']).\
                            group_by(['user_id', 'city_name']).aggregate([('request_cnt', "sum")]))\
                            .pivot(index='user_id', columns='city_name', values='request_cnt_sum')

In [22]:
for name in day_city_name_pl.columns:   
    day_city_name_pl = day_city_name_pl.with_columns(pl.col(name).cast(pl.Int32))

In [23]:
day_city_name_pl = day_city_name_pl.fill_null(0)

In [24]:
day_city_name_pl

user_id,Краснодар,Ставрополь,Адыгейск,Изобильный,Новоалександровск,Санкт-Петербург,Калининград,Новороссийск,Тюмень,Екатеринбург,Нефтеюганск,Пыть-Ях,Каменск-Уральский,Тобольск,Камышлов,Пермь,Челябинск,Копейск,Мурманск,Северск,Томск,Лесосибирск,Енисейск,Красноярск,Шарыпово,Назарово,Иланский,Ачинск,Новосибирск,Юрга,Горно-Алтайск,Обь,Набережные Челны,Нижнекамск,Новокуйбышевск,Самара,...,Вуктыл,Каменногорск,Спас-Клепики,Куртамыш,Александровск,Тетюши,Дальнегорск,Злынка,Нарткала,Устюжна,Урай,Юрьевец,Сенгилей,Николаевск-на-Амуре,Надым,Пыталово,Спас-Деменск,Асино,Онега,Южа,Урюпинск,Нолинск,Углегорск,Сурск,Эртиль,Ивангород,Темников,Мглин,Короча,Макаров,Наволоки,Бирюсинск,Дмитровск,Комсомольск,Никольск,Каргополь,Тара
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,...,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
45098,2254,4,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
117132,0,0,0,0,0,1286,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
79395,0,0,0,0,0,3167,139,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
91294,0,0,0,0,0,0,0,2437,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
161323,0,0,0,0,0,0,0,0,439,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
304574,0,0,0,0,0,0,0,0,0,0,197,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
204489,0,0,0,0,0,0,0,0,0,373,0,0,641,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
148545,0,0,0,0,0,0,0,0,0,124,0,0,0,4906,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16874,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2849,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
day_model_name_pl = pl.from_arrow(read_arrow.select(['user_id', 'cpe_model_name', 'request_cnt']).\
                            group_by(['user_id', 'cpe_model_name']).aggregate([('request_cnt', "sum")]))\
                            .pivot(index='user_id', columns='cpe_model_name', values='request_cnt_sum')

In [26]:
for name in day_model_name_pl.columns:   
    day_model_name_pl = day_model_name_pl.with_columns(pl.col(name).cast(pl.Int32))

In [27]:
day_model_name_pl = day_model_name_pl.fill_null(0)

In [28]:
day_model_name_pl

user_id,iPhone 7,Redmi 5 Plus,Galaxy S20+,Poco X3 Pro,Redmi Note 8 Pro,Honor 10,Galaxy A6 Plus 2018 Dual,Galaxy A52,Galaxy A40 Dual,iPhone 11 Pro,P Smart Z,Honor 10 Lite,Honor 20 Pro,Honor 9 Lite,iPhone 6S,Galaxy A5 2017 Dual,iPhone XR,Honor 7S Dual,Redmi 8A,Redmi 8,Honor 10X Lite,iPhone X,Honor 7X,iPhone 12 Pro Max,iPhone 11 Pro Max,Galaxy A30 Dual,Galaxy A6 2018 Dual,iPhone 8 Plus,Galaxy A51 Dual,Y5 Lite,Redmi Note 9 Pro,6i,iPhone 7 Plus,Redmi 7A,iPhone 11,Galaxy A10 Dual,...,S68Pro,BV9600 Pro,G20,Mi Mix 3,A74,ZenFone 6,SPARK 4,MI MIX2,Y5II 3G Dual,Blade A610,2 Dual,ZenFone 3 ZE520KL,Galaxy Note 3 4G,F5 Youth,Galaxy S4,Redmi Note 8 2021,3.1 Plus,Huawei Mate 40 Pro,16,Moto G9 Plus,Blade A6 Lite,Galaxy Fold,Galaxy A03 Core,Honor 8A Pro,Pixel 4,Xiaomi Poco M4 PRO,2.2,BV9500,Galaxy S7,Pixel 5,V430,9 5G,BV6100,Moto G5S Dual,GT Master,Galaxy Tab A7 10.4 2020,Power Five Max
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,...,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
45098,2261,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
117132,0,1286,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
79395,0,0,3306,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
91294,0,0,0,2437,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
161323,0,0,0,0,441,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
304574,0,0,0,0,0,198,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
204489,0,0,0,0,0,0,1014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
148545,0,0,0,0,0,0,0,5033,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16874,0,0,0,0,0,0,0,0,2849,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
data = data_cat_pl.join(day_part_pl, left_on='user_id', right_on='user_id')

In [30]:
data = data.join(day_region_name_pl, left_on='user_id', right_on='user_id')

In [31]:
data = data.join(day_city_name_pl, left_on='user_id', right_on='user_id')

In [32]:
data = data.join(day_model_name_pl, left_on='user_id', right_on='user_id')

In [33]:
data = data.join(data_date_activity, left_on='user_id', right_on='user_id')

In [34]:
data.write_parquet('data_category.parquet') 